# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_DFT(lattice, atoms, positions;
                  functionals=LDA(), symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770326127996                   -0.53    9.0    166ms
  2   -2.771687116006       -2.87       -1.30    1.0   92.9ms
  3   -2.771714234053       -4.57       -2.62    1.0   95.9ms
  4   -2.771714687990       -6.34       -3.96    1.0   97.1ms
  5   -2.771714714913       -7.57       -4.31    2.0    115ms
  6   -2.771714715237       -9.49       -5.54    1.0    486ms
  7   -2.771714715249      -10.92       -5.80    2.0    120ms
  8   -2.771714715250      -13.22       -5.94    2.0    147ms
  9   -2.771714715250      -12.97       -7.22    1.0    135ms
 10   -2.771714715250      -13.81       -7.43    2.0    115ms
 11   -2.771714715250      -13.86       -8.13    1.0    103ms


-0.00013457372923182838

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770478647853                   -0.53    9.0    229ms
  2   -2.771776605378       -2.89       -1.32    1.0    149ms
  3   -2.771801714333       -4.60       -2.51    1.0   93.3ms
  4   -2.771802009659       -6.53       -3.29    1.0   96.2ms
  5   -2.771802072460       -7.20       -3.85    2.0    123ms
  6   -2.771802074366       -8.72       -4.64    1.0    102ms
  7   -2.771802074464      -10.01       -4.99    2.0    543ms
  8   -2.771802074476      -10.92       -6.22    1.0    116ms
  9   -2.771802074476   +  -13.30       -5.82    2.0    153ms
 10   -2.771802074476      -12.56       -7.17    1.0    127ms
 11   -2.771802074476      -14.88       -7.31    2.0    117ms
 12   -2.771802074476      -13.97       -7.85    1.0    113ms
 13   -2.771802074476   +  -14.21       -8.63    1.0    106ms


0.017612221333693577

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457372923182838
Displaced dipole:  0.017612221333693577
Polarizability :   1.7746795062925402


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920641874e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551188486e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852766602181e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694593759000e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088787361622e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.273467781832e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 6.516603448243e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 6.516603448243e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.339715829327e-09
[ Info: GMRES linsolve in iter 2; step 2: normres = 3.205168544015e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 4.611492550780e-12
[ Info: GMRES linsolve in iter 2; step 4: normres = 5.695452170693e-14
[ Info: GMRES linsolve in iter 2; finished at step 4: normres = 5.695452170693e-14
┌ Info: GMRES linsolve converged at iteration 2, step

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.